In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_324912/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.8)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_9_16,0.875686,0.481823,0.856471,0.792760,0.834244,0.042322,0.307612,0.066442,0.065684,0.066063,0.081151,0.205724,0.778011,0.207377,116.324874,272.320379,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
1,model_1_9_15,0.875638,0.479154,0.856562,0.792695,0.834272,0.042339,0.309196,0.066399,0.065705,0.066052,0.081177,0.205764,0.777925,0.207417,116.324098,272.319603,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
2,model_1_9_18,0.875442,0.482977,0.854142,0.789224,0.831487,0.042406,0.306927,0.067520,0.066805,0.067162,0.081077,0.205926,0.777575,0.207580,116.320947,272.316452,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
3,model_1_9_20,0.875347,0.479898,0.852636,0.787653,0.829988,0.042438,0.308755,0.068217,0.067303,0.067760,0.080894,0.206005,0.777405,0.207660,116.319422,272.314927,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
4,model_1_9_19,0.875298,0.480525,0.852689,0.787768,0.830064,0.042455,0.308383,0.068192,0.067266,0.067729,0.080893,0.206045,0.777317,0.207700,116.318634,272.314139,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
5,model_1_9_17,0.875294,0.481194,0.855094,0.788586,0.831786,0.042456,0.307985,0.067079,0.067007,0.067043,0.081106,0.206048,0.777311,0.207703,116.318578,272.314083,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
6,model_1_9_14,0.875283,0.482621,0.856866,0.791752,0.834074,0.042460,0.307138,0.066259,0.066004,0.066131,0.081423,0.206058,0.777291,0.207713,116.318396,272.313901,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
7,model_1_9_21,0.875146,0.462751,0.849473,0.788855,0.828628,0.042506,0.318934,0.069681,0.066922,0.068302,0.080903,0.206171,0.777046,0.207827,116.316200,272.311705,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
8,model_1_9_13,0.874904,0.484013,0.858201,0.790636,0.834404,0.042589,0.306312,0.065641,0.066357,0.065999,0.081677,0.206370,0.776615,0.208028,116.312333,272.307838,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
9,model_1_9_12,0.874191,0.483943,0.860821,0.790659,0.835936,0.042832,0.306354,0.064428,0.066350,0.065389,0.082026,0.206958,0.775341,0.208620,116.300959,272.296464,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
